<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is available


In [59]:
import io
#df = pd.read_csv('Apple60.csv', skiprows=0)

from sklearn.model_selection import train_test_split
df = pd.read_csv('Para_News60_Combined60.csv', sep=',', header=0)
df["Movement"].replace({-1: 0}, inplace=True)
data = df.loc[df['Stock'] == 'Apple']
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [34]:
data[:10]

,Source,DateTime,News,Movement,Stock
0,www.businesswire.com,2017-12-07 20:00:00+00:00,NAPE Summit Week will also feature the annual ...,1,Apple
1,seekingalpha.com,2017-12-14 11:42:00+00:00,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,0,Apple
2,seekingalpha.com,2017-12-14 12:31:00+00:00,The areas to be concerned are that there are f...,1,Apple
3,seekingalpha.com,2017-12-15 12:04:00+00:00,Amazon will soon resume selling the Apple TV (...,1,Apple
5,seekingalpha.com,2017-12-18 20:05:00+00:00,"With such a large revenue pie, naturally we be...",0,Apple
7,seekingalpha.com,2017-12-21 12:04:00+00:00,"Facing questions from tech analysts, Apple (NA...",0,Apple
9,www.thestreet.com,2017-12-21 12:27:00+00:00,Cramer and the AAP team say investors need to ...,0,Apple
10,seekingalpha.com,2017-12-22 11:51:00+00:00,One day after Apple acknowledged that it has b...,0,Apple
11,www.thestreet.com,2017-12-26 13:48:00+00:00,1 . -- U.S. stock futures turned lower on Tues...,0,Apple
13,seekingalpha.com,2017-12-27 11:43:00+00:00,Shares in Apple and several of its suppliers f...,0,Apple


In [22]:
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tqdm import tqdm
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

MAX_NB_WORDS = 40000
max_seq_len = 1000

raw_docs_train = train_df['News'].tolist()
raw_docs_test = test_df['News'].tolist()

print(raw_docs_train[:10])

num_classes = 2
processed_docs_train = []

stop_words = set(stopwords.words('english'))

for doc in tqdm(raw_docs_train):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_train.append(" ".join(filtered))
processed_docs_test = []

for doc in tqdm(raw_docs_test):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_test.append(" ".join(filtered))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...


  0%|          | 50/17276 [00:00<00:38, 450.89it/s]

[nltk_data]   Package punkt is already up-to-date!
['For much of the past year, the trade war has disrupted the flow of hundreds of billions of dollars worth of goods and hit the global economy. Official data this week showed manufacturing activity slowed in both countries, and companies such as Apple Inc ( AAPL.O ) and Cargill Inc [CARG.UL] said the trade battle had hit earnings.', "U.S. stocks fell on Friday, with the S&P 500 returning to negative territory for the year as losses accelerated in midday trading. The benchmark index SPX, -0.85% lost 0.8%, and is now down less than 0.1% for 2018 thus far. While the S&P has been negative for the year multiple times in 2018, it had recently returned to the green. It has traded in positive territory for the year for the past four sessions. The Dow Jones Industrial Average DJIA, -0.82% fell 0.8% on Friday, bringing its year-to-date move to a drop of 1%. The Nasdaq Composite Index COMP, -1.27% fell 1.1% on Friday, but remains up 3.7% for the 

100%|██████████| 4319/4319 [00:07<00:00, 606.05it/s]


In [35]:
test_data = pd.read_csv('Apple60_Major.csv', sep=',', header=0)

test_corpus = test_data['News'].tolist()

processed_test_sample = []
for doc in tqdm(test_corpus):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_test_sample.append(" ".join(filtered))

100%|██████████| 711/711 [00:00<00:00, 1169.95it/s]


In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(raw_docs_train)

86037


In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  

vocab_length = len(tokenizer.word_index) + 1

print(vocab_length)

45952


In [37]:
MAX_NB_WORDS = vocab_length
max_seq_len = 1000

word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_seq_testsample = tokenizer.texts_to_sequences(processed_test_sample)
word_index = tokenizer.word_index

word_seq_train = pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = pad_sequences(word_seq_test, maxlen=max_seq_len)
word_seq_testsample = pad_sequences(word_seq_testsample, maxlen=max_seq_len)

print(len(word_index))

45951


In [13]:
import requests, zipfile, io
zip_file_url = "https://nlp.stanford.edu/data/glove.twitter.27B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print("GloVE available locally")

GloVE available locally


In [46]:
import codecs

embeddings_index = {}
#f = codecs.open(‘crawl-300d-2M.vec’, encoding=’utf-8')
# for Glove
f = codecs.open('glove.6B.100d.txt', encoding='utf-8')
for line in tqdm(f):
  values = line.rstrip().rsplit(' ')
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

400000it [00:17, 22650.18it/s]


In [47]:
words_not_found = []
embed_dim = 100
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
  if i >= nb_words:
     continue
  embedding_vector = embeddings_index.get(word)
  
  if (embedding_vector is not None) and len(embedding_vector) > 0:
     embedding_matrix[i] = embedding_vector
  else:
     words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 13310


In [48]:
from keras.layers import BatchNormalization
from keras.layers import Embedding, Bidirectional, Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D, Activation
from keras.initializers import Constant

model = tf.keras.Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix],trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 100)         4595200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 100)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 996, 64)           32064     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 249, 64)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 4,693,365
Trainable params: 98,165
Non-trainable params: 4,595,200
_______________________________________

In [49]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

labels = to_categorical(np.asarray(train_df['Movement']))
y_train = np.array(train_df['Movement'])

print(labels[:10])
print(y_train[:10])


[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[0 0 1 0 1 1 1 1 1 0]


In [50]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [54]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
es_callback = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(word_seq_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
432/432 [==============================] - 11s 21ms/step - loss: 0.3424 - accuracy: 0.8306 - f1_m: 0.8226 - precision_m: 0.8273 - recall_m: 0.8289 - val_loss: 1.0614 - val_accuracy: 0.5385 - val_f1_m: 0.5154 - val_precision_m: 0.5295 - val_recall_m: 0.5189
Epoch 2/10
432/432 [==============================] - 9s 20ms/step - loss: 0.3254 - accuracy: 0.8486 - f1_m: 0.8426 - precision_m: 0.8441 - recall_m: 0.8494 - val_loss: 1.0792 - val_accuracy: 0.5460 - val_f1_m: 0.5303 - val_precision_m: 0.5370 - val_recall_m: 0.5398
Epoch 3/10
432/432 [==============================] - 9s 20ms/step - loss: 0.3198 - accuracy: 0.8516 - f1_m: 0.8457 - precision_m: 0.8500 - recall_m: 0.8500 - val_loss: 1.1170 - val_accuracy: 0.5373 - val_f1_m: 0.5338 - val_precision_m: 0.5296 - val_recall_m: 0.5554
Epoch 4/10
432/432 [==============================] - 8s 20ms/step - loss: 0.3192 - accuracy: 0.8450 - f1_m: 0.8379 - precision_m: 0.8436 - recall_m: 0.8409 - val_loss: 1.1434 - val_accuracy: 0.5408

In [55]:
y_test1 = np.array(test_df['Movement'])
y_test2 = to_categorical(np.asarray(test_df['Movement']))
loss, accuracy, f1_score, precision, recall = model.evaluate(word_seq_test, y_test1, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss*100))
print('f1: %f' % (f1_score))
print('precision: %f' % (precision))
print('recall: %f' % (recall))

Accuracy: 55.753648
loss: 113.778436
f1: 0.557031
precision: 0.549021
recall: 0.577063


In [58]:
test_data["Movement"].replace({-1: 0}, inplace=True)
y_testsample = np.array(test_data['Movement'])
probas = model.predict(word_seq_testsample)
predict_labels = (probas < 0.5).astype(np.int)

print(predict_labels)
np.savetxt("foo.csv", predict_labels, delimiter=",")
loss, accuracy, f1_score, precision, recall = model.evaluate(word_seq_testsample, y_testsample, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss*100))
print('f1: %f' % (f1_score))
print('precision: %f' % (precision))
print('recall: %f' % (recall))

[[1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
